### Data Cleaning Exercise
Practice exercise to clean data in Python, using dummy data from the Boston Public School system. 

In [1]:
# import relevant Python packages
import pandas as pd
import numpy as np
import openpyxl

In [2]:
# read in relevant sheets from exercise file
sched = pd.read_excel('data/exercise.xlsx', 3)
roster = pd.read_excel('data/exercise.xlsx', 4)

There are duplicate records in the Roster sheet. Some duplicates seem to indicate multiple roles (e.g., Michael is both a Teacher and Social Worker), and potentially multiple races (e.g., Ivy is both White and Black). There is an ambiguous 'Empl Record' column I do not feel confident enough to use, as the Glossary doesn't specify how to interpret its values. Since this exercise is only concerned with job role, I am going to drop the Race and Sex columns, as well as the Empl Record column. This should allow me to remove duplicates where the Employee ID, First/Last Name, and Job Code/Title are all the same, as my analysis doesn't require knowing whether the person has multiple sexes, races, or Empl Records. It will still leave duplicate IDs where the staff member has multiple roles (e.g., Teacher and Social Worker) so I can categorize the person's role to the best of my knowledge later on. 

You can use the 'subset' parameter in drop_duplicates to specify which columns to ignore in evaluating duplicates (instead of dropping them completing). However, in this method, there's no way to know whether a row had a duplicate that's no longer in the dataset, so running this function renders those columns unusable. It's less confusing to drop completely. 

In [3]:
roster = roster[['ID', 'First Name', 'Last Name', 'Job Code', 'Job Title']]
roster = roster.drop_duplicates().reset_index(drop=True)

Each school has multiple classes, which can have multiple staff assigned to them, and staff can have multiple roles in the roster. 

Given that the task is to assign roles based on whether the staff member is serving alone, and the same class number is used at different schools, I assume that one should differentiate which duplicate classes are taught at the same school (and have multiple staff working within the same classroom) and which are taught at different schools concurrently by staff not working in the same classroom. This assumes that one school cannot have two different instances of the same class delivered concurrently (e.g. two teachers assigned to the same class number in the same school are working in the same classroom as co-teachers, not the primary teachers of two separate classes teaching the same subject matter).

In [4]:
# create a unique ID column for classes to control for the same class concurrently taught in different schools
sched['Class ID'] = sched['School'] + '---' + sched['Class']

Because a teacher can have multiple roles, a left join to add in teacher roles will produce duplicate rows. If Michael is both a Social Worker and Substitute Teacher, it will produce two matches in the join, producing two rows for a course that, before the join, was only one row for one class. I will log how duplicate situations there are, so we can make some cleaning decisions that take into account duplicate roles. 

In [5]:
# count how many times each unique staff ID appears in the roster sheet
unique_ids, id_ct = np.unique(roster['ID'], return_counts=True)

In [6]:
# count how many times each unique class ID appears in the sched sheet
unique_classes, class_ct = np.unique(sched['Class ID'], return_counts=True)

To be able to apply the logic provided that determines whether the staff member is a primary teacher, co-teacher, or other support staff, the Role column from the roster sheet should be brought into the active schedule. However, the Employee ID column in the Roster table is not unique, as staff can hold multiple roles. While the Glossary suggests that the Employee Record column may distinguish whether the job title is active, there isn't a clear enough pattern in the column to rely on it. 

There are key organizational logic questions that need to be answered before this data can be confidently cleaned. Can staff hold multiple roles at the same time? Can they serve in two official capacities in the same classroom (e.g., serving as both a Social Worker and a Substitute Teacher within the same classroom)? If they can, which Job Title should be preferenced for the Role column (or should there be an additional category for "in two roles")? Do certain Job Titles only apply to certain types of classes (e.g., is a Coach role only relevant to certain classroom subjects)? Without that context, the most accurate cleaning method is to separate staff with ambiguous duplicate roles out from those without to be able to sort them out as unconfident matches.

I'm choosing to create new columns to surface the conditions for these discrepancies (as opposed to putting it directly as a condition later on) for a few reasons. New columns make it easier to cross-check if the code is working -- you can more easily identify if logic isn't working, if a condition column has unexpected values. Creating these columns will also make the final logic in the conditions more human readable, so a person trying to understand the code later on will see the organizational logic more easily.

In [7]:
# create a conditions column to identify if there are more than one staff assigned to a Class
multiple_staff= []
for c in sched['Class ID']:
    if c in unique_classes[class_ct==1]:
        multiple_staff.append('Single Staff')
    elif c in unique_classes[class_ct>1]:
        multiple_staff.append('Multiple Staff')
    else: 
        multiple_staff.append('Error')

sched['Staff Count'] = multiple_staff

In [8]:
# create a conditions column to identify if a staff person has more than one role
multiple_roles= []
for c in roster['ID']:
    if c in unique_ids[id_ct==1]:
        multiple_roles.append('Single Role')
    elif c in unique_ids[id_ct>1]:
        multiple_roles.append('Multiple Roles')
    else: 
        multiple_roles.append('Error')

roster['Role Count'] = multiple_roles

In [9]:
# create a conditions column to identify if a staff person holds the role of 'Teacher' in any of their roles
is_teacher = []
for i, t in enumerate(roster['ID']):
    count = sum(roster[roster['ID']==roster['ID'][i]]['Job Title'].eq('Teacher'))
    is_teacher.append(count)

roster['Is Teacher'] = is_teacher

With most of the conditions columns created, we now can do a merge. This merge will still produce duplicate rows when a person has multiple roles, but our earlier drop_duplicates will reduce the total amount to ensure it's only for staff members with multiple different roles. 

In [10]:
# join job title column to names in schedule sheet to enable conditional categorization
full_sched = pd.merge(sched, roster[['ID', 'Job Title', 'Role Count', 'Is Teacher']], 'left', left_on='Employee ID', right_on='ID')

There is one more conditions column we need to put in place, which is only possible after the merge has been completed: how many people with the role of 'Teacher' have been assigned to a class. Let's make that now. 

In [11]:
# create a conditions column to identify the total number of teachers assigned to a class
multiple_teachers = []
for i, c in enumerate(full_sched['Class ID']):
    count = sum(full_sched[full_sched['Class ID']==full_sched['Class ID'][i]]['Job Title'].eq('Teacher'))
    multiple_teachers.append(count)

full_sched['Teacher Count'] = multiple_teachers

Now we have all the conditions in place to run the logic to make a new 'Corrected Role' column that assigns a role whenever we can confidently know it. If I had more time (and assuming this script is expected to be reused in the future), I would optimize it to ensure the script isn't repeating itself in any way, and that it runs as efficiently and unambiguously as possible.  

In [12]:
# overwrite the Role column to match organizational logic: 
## if staff member is not a Teacher, their role is Other Support Staff; 
## if staff member has another Teacher assigned to their class, their role is Co-Teacher; 
## if staff member is the only Teacher assigned to their class, their role is Primary
role_corrected = []
for index, row in full_sched.iterrows():
    # if the staff member is not a teacher -- e.g. even if they have multiple roles, none of them are 'Teacher' exactly -- categorize them as Other Support Staff
    # this treats does not treat leave, substitutes, management (e.g. headmasters), etc. as 'teachers' even though they could be serving in that capacity
    # this does not correct for circumstances where there are no teachers assigned to a Class
    if row['Is Teacher']==0:
        role_corrected.append('Other Support Staff')
    # if multiple staff are assigned to the class, and the staff person holds multiple roles (one of them being a teacher), and there are multiple teachers present, 
    # or if the staff person's only role is a teacher, but there are others assigned to the class that hold multiple roles (one of them being a teacher),
    # it's not possible to know what role the staff person is in -- all teachers could be in the teacher role, and therefore they'd be 'co-teachers', 
    # or only one could be primary and the other teachers could be playing other roles
    # categorize them as 'Unknown Role'
    elif (row['Staff Count'] == 'Multiple Staff' and row['Role Count']=='Multiple Roles' and row['Teacher Count']>1) or (row['Role Count']=='Single Role' and row['Teacher Count']>1 and 'Multiple Roles' in list(full_sched[full_sched['Class ID']==full_sched['Class ID'][index]][full_sched['Is Teacher']==1]['Role Count'])):
        role_corrected.append('Unknown Role')
    # if multiple staff are assigned to the class, and there are more than one teacher assigned (all who have only one role, that being 'Teacher' exactly, as checked by previous conditions),
    # categorize them as 'Co-Teachers'
    elif row['Staff Count'] == 'Multiple Staff' and row['Teacher Count']>1:
        role_corrected.append('Co-Teacher')
    # if there is only one staff member assigned to the class, and they are a Teacher in any of their roles
    # or if there are multiple staff assigned to the class, but the staff member is the only Teacher assigned to the Class
    elif (row['Staff Count'] == 'Single Staff' and row['Is Teacher']==1) or (row['Staff Count'] == 'Multiple Staff' and row['Is Teacher']==1 and row['Teacher Count']==1):
        role_corrected.append('Primary')
    else:
        role_corrected.append('Error')

full_sched['Correct Role'] = role_corrected

C:\Users\hooda\AppData\Local\Temp\ipykernel_19092\4221368158.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  elif (row['Staff Count'] == 'Multiple Staff' and row['Role Count']=='Multiple Roles' and row['Teacher Count']>1) or (row['Role Count']=='Single Role' and row['Teacher Count']>1 and 'Multiple Roles' in list(full_sched[full_sched['Class ID']==full_sched['Class ID'][index]][full_sched['Is Teacher']==1]['Role Count'])):
C:\Users\hooda\AppData\Local\Temp\ipykernel_19092\4221368158.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  elif (row['Staff Count'] == 'Multiple Staff' and row['Role Count']=='Multiple Roles' and row['Teacher Count']>1) or (row['Role Count']=='Single Role' and row['Teacher Count']>1 and 'Multiple Roles' in list(full_sched[full_sched['Class ID']==full_sched['Class ID'][index]][full_sched['Is Teacher']==1]['Role Count'])):
C:\Users\hooda\AppData\Local\Temp\ipykernel_19092\4221368158.py:17: User

At this point, a new corrected role column has been created, as well as multiple conditions columns that allow the viewer to cross-check the result for accuracy in both executing my logic, as well as their own logic if their understanding is different than mine. I could consolidate this down by dropping and/or merging the Job Title column to remove the duplicate rows, but since I've made a lot of assumptions in my cleaning, I'd prefer to leave the duplicates to allow a viewer to fully cross-check. 

If I had more time, I would fully cross-check this result, which would probably include optimizing this code further to enable easier checks in the final result. If there was a source of truth (e.g., the system where these Excel sheets were pulled from), I would also cross-check my results against that standard at this point. 

In [13]:
# create, and write to, a new Excel file for the cleaned data
cleaned_book = openpyxl.Workbook('data/Data Exercise_AHood_Part 1.xlsx')
cleaned_sheet = cleaned_book.active

full_sched.to_excel('data/Data Exercise_AHood_Part 1.xlsx')

After going through this exercise, I'm also curious to know about how the source data is organized, and what opportunities exist for improving the base data model and ID system. Another (longer, less-immediate-results-focused) approach to this exercise would've been to seek to reorganize the tables into a star schema with unique IDs, and then create the results tables from there.